# Extracting and analysing data related to an AOP of interest

Citation: Marvin Martens, Thomas Exner, Tomaž Mohorič, Chris T Evelo, Egon L Willighagen. Workflow for extracting and analyzing data related to an AOP of interest. 2020

One of the main questions to solve in [AOPLink](https://openrisknet.org/e-infrastructure/development/case-studies/case-study-aoplink/) is the finding of data that supports an AOP of interest. To answer that, we have developed this Jupyter notebook that does that by using a variety of OpenRiskNet services:

- AOP-Wiki RDF
- ChemIdConvert
- AOP-DB RDF
- BridgeDb
- EdelweissData explorer
- WikiPathways

After selecting an AOP of interest, information is extracted from the AOP-Wiki RDF, ChemIdConvert, and AOP-DB RDF, to get a better understanding of the AOP. Next, the EdelweissData explorer was used to search for fitting data sets from ToxCast and TG-GATES based on the genes and compounds linked to the AOP of interest. The final part involves pathway analysis using the transcriptomics data of TG-GATES and the molecular pathways of WikiPathways, identifying significantly affected pathways upon exposure to the chemicals of interest.

In order to execute the Jupyter notebook, a set of Python libraries are required. The following section should import, or install, all of them.

In [1]:
import sys

!{sys.executable} -m pip install --upgrade pip 
!{sys.executable} -m pip install watermark

try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except ImportError:
    !{sys.executable} -m pip install sparqlwrapper
    from SPARQLWrapper import SPARQLWrapper, JSON
    
try:
    from pyvis.network import Network
except ImportError:
    !{sys.executable} -m pip install pyvis
    from pyvis.network import Network

try:
    from IPython.display import display, HTML, IFrame
except ImportError:
    !{sys.executable} -m pip install ipython
    from IPython.display import display, HTML, IFrame

try:
    import urllib
except ImportError:
    !{sys.executable} -m pip install urllib
    import urllib

try:
    import simplejson as json
except ImportError:
    !{sys.executable} -m pip install simplejson
    import simplejson as json

try:
    import pandas as pd
except ImportError:
    !{sys.executable} -m pip install pandas
    import pandas as pd
    
try:
    import re
except ImportError:
    !{sys.executable} -m pip install re
    import re

try:
    import requests
except ImportError:
    !{sys.executable} -m pip install requests
    import requests
    
try:
    import warnings
except ImportError:
    !{sys.executable} -m pip install warnings
    import warnings

try:
    import statistics
except ImportError:
    !{sys.executable} -m pip install statistics
    import statistics

try:
    from edelweiss_data import API, QueryExpression as Q
except ImportError:
    !{sys.executable} -m pip install edelweiss_data
    from edelweiss_data import API, QueryExpression as Q  
    
pd.set_option('display.max_colwidth', -1)
warnings.filterwarnings("ignore", category=UserWarning)

     |████████████████████████████████| 1.5MB 7.1MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 344 kB 8.7 MB/s 
     |████████████████████████████████| 67 kB 18.1 MB/s 
     |████████████████████████████████| 45 kB 20.4 MB/s 
     |████████████████████████████████| 1.6 MB 8.4 MB/s 
     |████████████████████████████████| 83 kB 6.7 MB/s 
  Created wheel for simplejson: filename=simplejson-3.17.0-cp36-cp36m-linux_x86_64.whl size=111916 sha256=9416e23f3d68d8c6a642e5994636621d06b1d445da8b925b0aa32cb163cec392
  Stored in directory: /tmp/pip-ephem-wheel-cache-v0ly96hg/wheels/c2/95/a2/477b46cfe980061fddc4cb76ed4657eceec35d2b2090355e41
Successfully built simplejson
     |████████████████████████████████| 10.4 MB 7.5 MB/s 
     |████████████████████████████████| 20.1 MB 24.2 MB/s 
     |████████████████████████████████| 509 kB 24.0 MB/s 


# Define the AOP of interest

This Jupyter notebook focuses on the AOP of interest, which is based on the identifier of the AOP. The identifiers of AOPs can be found on the [AOP-Wiki website](https://aopwiki.org/aops?direction=asc&sort=id "AOP-Wiki").

In [2]:

AOPid = "37"


# Set service URLs

The notebook uses a variety of external services. To keep an overview of these, their URLs are defined at the start of the notebook.

In [3]:
# SPARQL endpoint URLs
aopwikisparql = SPARQLWrapper("http://aopwiki-rdf.prod.openrisknet.org/sparql/")
aopdbsparql = SPARQLWrapper("http://aopdb-rdf.prod.openrisknet.org/sparql/")
wikipathwayssparql = SPARQLWrapper("http://sparql.wikipathways.org/sparql/")

# ChemIdConvert URL
chemidconvert = 'https://chemidconvert.cloud.douglasconnect.com/v1/'

# BridgeDB base URL
bridgedb = 'http://bridgedb.prod.openrisknet.org/'

# EdelweissData API URL
edelweiss_api_url = 'https://api.staging.kit.cloud.douglasconnect.com'

# AOP-Wiki RDF

## Service description
The AOP-Wiki repository is part of the AOP Knowledge Base (AOP-KB), a joint effort of the US-Environmental Protection Agency and European Commission - Joint Research Centre. It is developed to facilitate collaborative AOP development, storage of AOPs, and therefore allow reusing toxicological knowledge for risk assessors. This Case Study has converted the AOP-Wiki XML data into an RDF schema, which has been exposed in a public SPARQL endpoint in the OpenRiskNet e-infrastructure.

## Implementation
First, general information of the AOP is fetched using a variety of SPARQL queries, using predicates from the [AOP-Wiki RDF schema](https://doi.org/10.6084/m9.figshare.11323685.v1 "AOP-Wiki"). 
This is used for:
- Creating an overview table of the AOP of interest
- Extending the AOP network with connected AOPs

Second, stressor chemicals are retrieved and stored for further analysis and fetching of data.

## Creating overview table

In [4]:
#Define all variables as ontology terms present in AOP-Wiki RDF
title = 'dc:title'
webpage = 'foaf:page'
creator = 'dc:creator'
abstract = 'dcterms:abstract'
key_event = 'aopo:has_key_event'
molecular_initiating_event = 'aopo:has_molecular_initiating_event'
adverse_outcome = 'aopo:has_adverse_outcome'
key_event_relationship = 'aopo:has_key_event_relationship'
stressor = 'ncit:C54571'

#Create the list of all terms of interest
listofterms = [title,webpage,creator,abstract,key_event,molecular_initiating_event,adverse_outcome,key_event_relationship,stressor]

#Initiate the DataFrame
AOPinfo = pd.DataFrame(columns=['Properties'], index = [list(listofterms)])

#Query all terms of interest in the selected AOP
for term in listofterms:
    sparqlquery = '''
    PREFIX ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
    SELECT (group_concat(distinct ?item;separator=";") as ?items)
    WHERE{
    ?AOP_URI a aopo:AdverseOutcomePathway;'''+term+''' ?item.
    FILTER (?AOP_URI = aop:'''+AOPid +''')}
    '''
    aopwikisparql.setQuery(sparqlquery)
    aopwikisparql.setReturnFormat(JSON)  
    results = aopwikisparql.query().convert()
    for result in results["results"]["bindings"]:
        if 'identifiers.org' in result["items"]["value"]:
            AOPinfo.at[term,'Properties'] = ', '.join(result["items"]["value"].split(';'))
        else:
            AOPinfo.at[term,'Properties'] = result["items"]["value"]
display(AOPinfo)

,Properties
dc:title,PPARalpha-dependent liver cancer
foaf:page,http://identifiers.org/aop/37
dc:creator,"J. Christopher Corton, Cancer AOP Workgroup. National Health and Environmental Effects Research Laboratory, Office of Research and Development, Integrated Systems Toxicology Division, US Environmental Protection Agency, Research Triangle Park, NC. Corresponding author for wiki entry (corton.chris@epa.gov)\n"
dcterms:abstract,"Several therapeutic agents and industrial chemicals induce liver tumors in rats and mice through the activation of the peroxisome proliferator-activated receptor alpha (PPARa). The molecular and cellular events by which PPARa activators induce rodent hepatocarcinogenesis have been extensively studied and elucidated. The weight of evidence relevant to the hypothesized AOP for PPARa activator-induced rodent hepatocarcinogenesis is summarized here. Chemical-specific and mechanistic data support concordance of temporal and dose&ndash;response relationships for the key events associated with many PPARa activators including a phthalate ester plasticizer di(2-ethylhexyl)phthalate (DEHP) and the drug gemfibrozil. The key events (KE) identified are KE1&ndash;PPARa activation, KE2&ndash;alteration in cell growth pathways, KE3&ndash;perturbation of cell growth and survival including increases in cell proliferation and effects on apoptosis, KE4&ndash;selective clonal expansion of preneoplastic foci cells, which lead to the Adverse Outcome&ndash;increases in hepatocellular adenomas and carcinomas. A number of molecular events were identified which were initially evaluated as possible key events. However, the data were not convincing for these to be key events in the MOA. Rather, although not causal these modulating factors were considered to have the potential to alter the ability of PPARa activators to increase liver cancer. These modulating events include increases in oxidative stress, activation of NF-kB, and inhibition of gap junction intercellular communication. While biologically plausible in humans, the hypothesized key events in the rodent MOA, for PPARa activators, are unlikely to induce liver tumors in humans because of biological differences in responses. This conclusion is based on minimal or no effects observed on growth pathways, hepatocellular proliferation and liver tumors in humans and/or species (including hamsters, guinea pigs and cynomolgous monkeys) that are more appropriate human surrogates than mice and rats at overlapping dose levels.\n"
aopo:has_key_event,"http://identifiers.org/aop.events/1170, http://identifiers.org/aop.events/1171, http://identifiers.org/aop.events/227, http://identifiers.org/aop.events/716, http://identifiers.org/aop.events/719"
aopo:has_molecular_initiating_event,http://identifiers.org/aop.events/227
aopo:has_adverse_outcome,http://identifiers.org/aop.events/719
aopo:has_key_event_relationship,"http://identifiers.org/aop.relationships/1229, http://identifiers.org/aop.relationships/1230, http://identifiers.org/aop.relationships/1232, http://identifiers.org/aop.relationships/1239"
ncit:C54571,"http://identifiers.org/aop.stressor/11, http://identifiers.org/aop.stressor/175, http://identifiers.org/aop.stressor/191, http://identifiers.org/aop.stressor/205, http://identifiers.org/aop.stressor/206, http://identifiers.org/aop.stressor/207, http://identifiers.org/aop.stressor/208, http://identifiers.org/aop.stressor/210, http://identifiers.org/aop.stressor/211"


## Generating AOP network

In [5]:
#Generate network from AOP + interlinked AOPs.
Key_Events = str(AOPinfo.iat[4,0]).split(', ')

#From all the KEs from the selected, get the other AOPs in which they are present, and find all KEs of those other AOPs
for Key_Event in Key_Events:
    sparqlquery = '''
    SELECT ?MIE_ID ?KE_ID ?AO_ID ?KER_ID ?KE_Title
    WHERE{
    ?KE_URI a aopo:KeyEvent; dcterms:isPartOf ?AOP_URI.
    ?AOP_URI aopo:has_key_event ?KE_URI2; aopo:has_molecular_initiating_event ?MIE_URI; aopo:has_adverse_outcome ?AO_URI; aopo:has_key_event_relationship ?KER_URI.
    ?KE_URI2 rdfs:label ?KE_ID; dc:title ?KE_Title. 
    ?MIE_URI rdfs:label ?MIE_ID.
    ?AO_URI rdfs:label ?AO_ID.
    ?KER_URI rdfs:label ?KER_ID.    
    FILTER (?KE_URI = <'''+Key_Event+'''>)}
    '''
    aopwikisparql.setQuery(sparqlquery)
    aopwikisparql.setReturnFormat(JSON)  
    results = aopwikisparql.query().convert()

    MIEs = set([])
    KEs = set([])
    KEtitle = {}
    AOs = set([])
    KERs = set([])
    for result in results["results"]["bindings"]:
        MIEs.add(result["MIE_ID"]["value"])
        AOs.add(result["AO_ID"]["value"])
        KEs.add(result["KE_ID"]["value"])
        KERs.add(result["KER_ID"]["value"])
        KEtitle[result["KE_ID"]["value"]]=result["KE_Title"]["value"]

#List all intermediate KEs that are not MIEs or AOs
KEsIntermediate = []
for item in KEs:
    if item not in MIEs and item not in AOs:
        KEsIntermediate.append(item) 

#Initiate network figure
net= Network(height="100%", width="100%")

#Add nodes for Molecular Initiating Events, Key Events, and Adverse Outcomes to the network figure
for MIE in MIEs:
    net.add_node(MIE, color = 'lightgreen', size = 50, shape = 'circle', font = '20px arial black', title = KEtitle[MIE])
for KE in KEsIntermediate:
    net.add_node(KE, color = 'khaki', size = 50, shape = 'circle', font = '20px arial black', title = KEtitle[KE])
for AO in AOs:
    net.add_node(AO, color = 'salmon', size = 50, shape = 'circle', font = '20px arial black', title = KEtitle[AO])

#Add all Key Event Relationships to the network figure after querying all KERs for all KEs in AOP-Wiki RDF
for KER in KERs:
    sparqlquery = '''
    SELECT ?KE_UP_ID ?KE_DOWN_ID 
    WHERE{
    ?KER_URI a aopo:KeyEventRelationship; rdfs:label ?KER_ID; aopo:has_upstream_key_event ?KE_UP_URI; aopo:has_downstream_key_event ?KE_DOWN_URI.
    ?KE_UP_URI rdfs:label ?KE_UP_ID.
    ?KE_DOWN_URI rdfs:label ?KE_DOWN_ID.
    FILTER (?KER_ID = "'''+KER+'''")}
    '''
    aopwikisparql.setQuery(sparqlquery)
    aopwikisparql.setReturnFormat(JSON)  
    results = aopwikisparql.query().convert()
    for result in results["results"]["bindings"]:
        net.add_edge(result["KE_UP_ID"]["value"],result["KE_DOWN_ID"]["value"],width = 2, color = 'black',label = KER, arrows = 'to')

net.show('mygraph.html')
IFrame(src='./mygraph.html', width=700, height=600)

## Query all chemicals that are part of the selected AOP

In [6]:
sparqlquery = '''
PREFIX ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
SELECT ?CAS_ID (fn:substring(?CompTox,33) as ?CompTox_ID) ?Chemical_name
WHERE{
?AOP_URI a aopo:AdverseOutcomePathway; ncit:C54571 ?Stressor.
?Stressor aopo:has_chemical_entity ?Chemical.
?Chemical cheminf:CHEMINF_000446 ?CAS_ID; dc:title ?Chemical_name.
OPTIONAL {?Chemical cheminf:CHEMINF_000568 ?CompTox.}
FILTER (?AOP_URI = aop:'''+AOPid +''')}
'''
aopwikisparql.setQuery(sparqlquery)
aopwikisparql.setReturnFormat(JSON)  
results = aopwikisparql.query().convert()

Chemical_names = {}
CompTox = {}

for result in results["results"]["bindings"]:
    try: CompTox[result["CAS_ID"]["value"]] =result["CompTox_ID"]["value"]
    except: pass
for result in results["results"]["bindings"]:
    try: Chemical_names[result["CAS_ID"]["value"]] =result["Chemical_name"]["value"]
    except: pass

Chemdata = pd.DataFrame(columns=['Chemical_name','CAS_ID','CompTox_ID'])
for CAS_ID in Chemical_names:
    Chemdata = Chemdata.append({
        'Chemical_name' : Chemical_names[CAS_ID],
        'CAS_ID'        : CAS_ID,
        'CompTox_ID'    : CompTox[CAS_ID],
        }, ignore_index=True)
display(Chemdata)

,Chemical_name,CAS_ID,CompTox_ID
0,Di(2-ethylhexyl) phthalate,117-81-7,DTXSID5020607
1,Gemfibrozil,25812-30-0,DTXSID0020652
2,Nafenopin,3771-19-5,DTXSID8020911
3,Bezafibrate,41859-67-0,DTXSID3029869
4,Fenofibrate,49562-28-9,DTXSID2029874
5,Pirinixic acid,50892-23-4,DTXSID4020290
6,Ciprofibrate,52214-84-3,DTXSID8020331
7,Clofibrate,637-07-0,DTXSID3020336


In [7]:
compounds = []
for index, row in Chemdata.iterrows():
    compounds.append(row['CAS_ID'])
compounds

['117-81-7',
 '25812-30-0',
 '3771-19-5',
 '41859-67-0',
 '49562-28-9',
 '50892-23-4',
 '52214-84-3',
 '637-07-0']

# ChemIdConvert

## Service description
The ChemIdConverter allows users to submit and translate a variety of chemical descriptors, such as SMILES and InChI, through a REST API.

## Implementation
Convert selected chemical names and display their chemical structures in a dataframe. It takes CAS IDs as an input, and translates them into Smiles and InChI Keys. 


In [8]:
compoundstable = pd.DataFrame(columns=['CAS_ID', 'Image', 'Smiles', 'InChIKey'])

# Fill "compounds" with the "smiles" by the compound name.
for compound in compounds:
    smiles = requests.get(chemidconvert + 'cas/to/smiles', params={'cas': compound}).json()['smiles']
    inchikey = requests.get(chemidconvert + 'smiles/to/inchikey', params={'smiles': smiles}).json()['inchikey']
    compoundstable = compoundstable.append({'CAS_ID': compound, 'Image': smiles, 'Smiles': smiles, 'InChIKey' : inchikey}, ignore_index=True)

def smiles_to_image_html(smiles):  # "smiles" shadows "smiles" from outer scope, use this function only in "to_html().
    """Gets for each smile the image, in HTML.
    :param smiles: Takes the “smiles” form “compounds”.
    :return: The HTML code for the image of the given smiles.
    """
    return '<img style="width:150px" src="' + chemidconvert+'asSvg?smiles='+urllib.parse.quote(smiles)+'"/>'


# Return a HTML table of "compounds", after "compounds" is fill by "smiles_to_image_html".
HTML(compoundstable.to_html(escape=False, formatters=dict(Image=smiles_to_image_html)))

,CAS_ID,Image,Smiles,InChIKey
0,117-81-7,,CCCCC(CC)COC(=O)c1ccccc1C(=O)OCC(CC)CCCC,BJQHLKABXJIVAM-UHFFFAOYSA-N
1,25812-30-0,,Cc1ccc(C)c(OCCCC(C)(C)C(O)=O)c1,HEMJJKBWTPKOJG-UHFFFAOYSA-N
2,3771-19-5,,CC(C)(Oc1ccc(cc1)C2CCCc3ccccc23)C(O)=O,XJGBDJOMWKAZJS-UHFFFAOYSA-N
3,41859-67-0,,CC(C)(Oc1ccc(CCNC(=O)c2ccc(Cl)cc2)cc1)C(O)=O,IIBYAHWJQTYFKB-UHFFFAOYSA-N
4,49562-28-9,,CC(C)OC(=O)C(C)(C)Oc1ccc(cc1)C(=O)c2ccc(Cl)cc2,YMTINGFKWWXKFG-UHFFFAOYSA-N
5,50892-23-4,,Cc1cccc(Nc2cc(Cl)nc(SCC(O)=O)n2)c1C,SZRPDCCEHVWOJX-UHFFFAOYSA-N
6,52214-84-3,,CC(C)(Oc1ccc(cc1)C2CC2(Cl)Cl)C(O)=O,KPSRODZRAIWAKH-UHFFFAOYSA-N
7,637-07-0,,CCOC(=O)C(C)(C)Oc1ccc(Cl)cc1,KNHUKKLJHYUCFP-UHFFFAOYSA-N


# AOP-DB RDF

## Service description
The EPA AOP-DB supports the discovery and development of putative and potential AOPs. Based on public annotations, it integrates AOPs with gene targets, chemicals, diseases, tissues, pathways, species orthology information, ontologies, and gene interactions. The AOP-DB facilitates the translation of AOP biological context, and associates assay, chemical and disease endpoints with AOPs ([Pittman et al., 2018](https://doi.org/10.1016/j.taap.2018.02.006); [Mortensen et al., 2018)](https://doi.org/10.1007/s00335-018-9738-7). The AOP-DB won the first OpenRiskNet implementation challenge of the associated partner program and is therefore integrated into the OpenRiskNet e-infrastructure. After the conversion of the AOP-DB into an RDF schema, its data will be exposed in a Virtuoso SPARQL endpoint.

## Implementation
Extract all genes related to AOP of interest
Find all ToxCast assays linked to those genes

In [9]:
Key_Events = str(AOPinfo.iat[4,0]).split(', ')
Genes = []
#from the KEs, get the AOPs
for Key_Event in Key_Events:
    sparqlquery = '''
    SELECT DISTINCT ?KE_ID ?Entrez_ID WHERE{
    ?KE_URI edam:data_1027 ?Entrez_URI. ?Entrez_URI edam:data_1027 ?Entrez_ID.
    FILTER (?KE_URI = <'''+Key_Event +'''>)}
    '''
    aopdbsparql.setQuery(sparqlquery)
    aopdbsparql.setReturnFormat(JSON)  
    results = aopdbsparql.query().convert()
    for result in results["results"]["bindings"]:
        Genes.append(result["Entrez_ID"]["value"])
print(Genes)

['5465', '403654', '19013', '25747']


# BridgeDb to map identifiers

## Service description
In order to link databases and services that use particular identifiers for genes, proteins, and chemicals, the BridgeDb platform is integrated into the OpenRiskNet e-infrastructure. It allows for identifier mapping between various biological databases for data integration and interoperability ([van Iersel et al., 2010](https://doi.org/10.1186/1471-2105-11-5)).

## Implementation
The genes from AOP-DB are mapped to identifiers from other databases using BridgeDb. Variable values are filled for ```inputdatasource``` and ```outputdatasource``` identifiers based on BridgeDb's [documentation on system codes](https://www.bridgedb.org/documentation/system-codes/). Also, the species is specified as a value in the variable ```Species```.


In [10]:
inputdatasource = 'L'
outputdatasource = ['H','En']
Species = ['Human','Dog','Mouse','Rat']
Mappings = {}
HGNC = []

for source in outputdatasource:
    Mappings[source] = []
    for Entrez in Genes:
        for species in Species:
            allmappings = re.split('\t|\n', requests.get(bridgedb + species + '/xrefs/' + inputdatasource + '/' + Entrez + '?dataSource='+source).text)
            if allmappings[0] is not '':
                break
        Mappings[source].append(allmappings[0])

ids = {}
for source in Mappings:
    ids[source]=[]
    for identifier in Mappings[source]:
        ids[source].append(identifier)

GenesTable = pd.DataFrame(columns=['Entrez','HGNC','Ensembl'])
GenesTable['Entrez'] = Genes
GenesTable['HGNC'] = ids['H']
GenesTable['Ensembl'] = ids['En']

display(GenesTable)

,Entrez,HGNC,Ensembl
0,5465,PPARA,ENSG00000186951
1,403654,,ENSCAFG00000000788
2,19013,,ENSMUSG00000022383
3,25747,,ENSRNOG00000021463


# EdelweissData explorer

Curated datasets are made available through the EdelweissData Explorer, the main data provisioning tool in the [DataCure case study](https://openrisknet.org/e-infrastructure/development/case-studies/case-study-datacure/) of OpenRiskNet. It is a web-based data explorer tool that gives users the ability to filter, search and extract data through the use of API calls. The EdelweissData Explorer serves data from ToxCast, ToxRefDB, and TG-GATES. 

Prior to using the EdelweissData explorer, the EdelweissData library is initialized and authenticated.


In [11]:
api = API(edelweiss_api_url)
api.authenticate()

## Tox21/ToxCast
Based on the identified genes and chemicals of interest by the AOP-DB RDF and AOP-Wiki RDF, the EdelweissData library is used to find datasets with those particular target genes and chemicals. 

First, the assays are searched based on the Entrez gene IDs. These are, along with their metadata, stored in a dataframe.

Next, for those assays, all datasets are retrieved that are generated with the compounds related to the AOP.

In [12]:
columns = [
#    ("Endpoint", "$.assay.component.endpoint"),
    ("Endpoint name", "$.assay.component.endpoint.assay_component_endpoint_name.value"),
    ("Biological target", "$.assay.component.endpoint.target.biological_process_target.value"),
    ("Entrez gene ID for the molecular target", "$.assay.component.endpoint.target.intended.intended_target_gene.intended_target_entrez_gene_id.value"),
    ("Symbol", "$.assay.component.endpoint.target.intended.intended_target_gene.intended_target_official_symbol.value"),
    ("Gene name", "$.assay.component.endpoint.target.intended.intended_target_gene.intended_target_gene_name.value")]

cquery = None
for gene in Genes:
    if cquery is None:
        cquery = Q.search_anywhere("EPA-InVitroDBV3.2") & Q.search_anywhere("summary") & Q.exact_search(Q.column('Entrez gene ID for the molecular target'), gene)
    else:
        cquery = cquery | Q.search_anywhere("EPA-InVitroDBV3.2") & Q.search_anywhere("summary") & Q.exact_search(Q.column('Entrez gene ID for the molecular target'), gene)

ToxCast = api.get_published_datasets(limit=200, columns=columns, condition=cquery)
ToxCast

,,dataset,Endpoint name,Biological target,Entrez gene ID for the molecular target,Symbol,Gene name
id,version,,,,,,
d651ef92-c12e-4eba-8979-8dc3f77fc7f3,1,<PublishedDataset 'd651ef92-c12e-4eba-8979-8dc3f77fc7f3':1 - EPA-InVitroDBV3.2-ATG_PPARa_TRANS_dn_summary_tcpl>,ATG_PPARa_TRANS_dn,regulation of transcription factor activity,5465,PPARA,peroxisome proliferator-activated receptor alpha
5a9cf864-520d-4ca0-b77f-8333aa8a3d5c,1,<PublishedDataset '5a9cf864-520d-4ca0-b77f-8333aa8a3d5c':1 - EPA-InVitroDBV3.2-ATG_PPRE_CIS_dn_summary_tcpl>,ATG_PPRE_CIS_dn,regulation of transcription factor activity,5465,PPARA,peroxisome proliferator-activated receptor alpha
cd27c421-8273-41cb-9e11-8c96a2b30e10,1,<PublishedDataset 'cd27c421-8273-41cb-9e11-8c96a2b30e10':1 - EPA-InVitroDBV3.2-NVS_NR_hPPARa_summary_tcpl>,NVS_NR_hPPARa,receptor binding,5465,PPARA,peroxisome proliferator-activated receptor alpha
291d8013-662b-4b44-aa96-894be4473d59,1,<PublishedDataset '291d8013-662b-4b44-aa96-894be4473d59':1 - EPA-InVitroDBV3.2-ATG_PPRE_CIS_up_summary_tcpl>,ATG_PPRE_CIS_up,regulation of transcription factor activity,5465,PPARA,peroxisome proliferator-activated receptor alpha
f3ebdf70-976f-4b19-92a1-25b189fa13fa,1,<PublishedDataset 'f3ebdf70-976f-4b19-92a1-25b189fa13fa':1 - EPA-InVitroDBV3.2-ATG_PPARa_TRANS_up_summary_tcpl>,ATG_PPARa_TRANS_up,regulation of transcription factor activity,5465,PPARA,peroxisome proliferator-activated receptor alpha


In [13]:
cquery = None
for compound in compoundstable['InChIKey'].values:
    if cquery is None:
        cquery = Q.fuzzy_search(Q.column('InChI key'), compound)
    else:
        cquery = cquery | Q.fuzzy_search(Q.column('InChI key'), compound)

ToxCastData = pd.DataFrame()
for index, row in ToxCast.iterrows():
    tmpdata = row['dataset'].get_data(limit=None, condition=cquery)
    tmpdata['Assay']=row['Endpoint name']
    tmpdata = tmpdata[['Assay', 'DTXSID', 'Substance name', 'InChI key', 'CAS', 'IC50', 'Quality check']]
    ToxCastData = pd.concat([ToxCastData, tmpdata])
    
ToxCastData.sort_values(by=['InChI key','Assay'])

,Assay,DTXSID,Substance name,InChI key,CAS,IC50,Quality check
3714,ATG_PPARa_TRANS_dn,DTXSID5020607,Di(2-ethylhexyl) phthalate,BJQHLKABXJIVAM-UHFFFAOYSA-N,117-81-7,NaN,[]
3714,ATG_PPARa_TRANS_up,DTXSID5020607,Di(2-ethylhexyl) phthalate,BJQHLKABXJIVAM-UHFFFAOYSA-N,117-81-7,NaN,[Borderline inactive]
3714,ATG_PPRE_CIS_dn,DTXSID5020607,Di(2-ethylhexyl) phthalate,BJQHLKABXJIVAM-UHFFFAOYSA-N,117-81-7,NaN,[]
3714,ATG_PPRE_CIS_up,DTXSID5020607,Di(2-ethylhexyl) phthalate,BJQHLKABXJIVAM-UHFFFAOYSA-N,117-81-7,1.414849,[]
24,NVS_NR_hPPARa,DTXSID5020607,Di(2-ethylhexyl) phthalate,BJQHLKABXJIVAM-UHFFFAOYSA-N,117-81-7,NaN,[]
919,ATG_PPARa_TRANS_dn,DTXSID0020652,Gemfibrozil,HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,NaN,[]
919,ATG_PPARa_TRANS_up,DTXSID0020652,Gemfibrozil,HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,1.558449,[]
919,ATG_PPRE_CIS_dn,DTXSID0020652,Gemfibrozil,HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,NaN,[]
919,ATG_PPRE_CIS_up,DTXSID0020652,Gemfibrozil,HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,1.509815,[]
3245,ATG_PPARa_TRANS_dn,DTXSID3029869,Bezafibrate,IIBYAHWJQTYFKB-UHFFFAOYSA-N,41859-67-0,NaN,[Noisy data]


## TG-GATES

Based on the chemicals of interest, all transcriptomics datasets from TG-GATES are queried that are the result of exposure to those chemicals.

In [14]:
columns = [
    ("Compound", "$.Compound.Name"),
    ("InChI Key", "$.Compound.\"InChI Key\""),
    ("CAS", "$.Compound.CAS"),
    ("Organism", "$.Assay.Organism"),
    ("Organ", "$.Assay.Organ"),
    ("Study type", "$.Assay.\"Study type\""),
    ("Dose", "$.Assay.Exposure.Dose"),
    ("Dosing", "$.Assay.Dosing"),
    ("Duration", "$.Assay.Exposure.Duration"),
    ("Duration unit", "$.Assay.Exposure.\"Duration unit\""),
]

cquery = None
for compound in compoundstable['InChIKey'].values:
    if cquery is None:
        cquery = Q.fuzzy_search(Q.column('InChI Key'), compound)
    else:
        cquery = cquery | Q.fuzzy_search(Q.column('InChI Key'), compound)

condition = Q.search_anywhere("TG-GATES") & Q.search_anywhere("FOLD_CHANGES") & (cquery)
TGGATES = api.get_published_datasets(limit=api.get_raw_datasets(limit=0, columns=columns, condition=condition)['total'], columns=columns, condition=condition)
TGGATES['Duration2'] = TGGATES['Duration'].map(str) + TGGATES['Duration unit']
TGGATES['Duration'] = TGGATES['Duration2']
TGGATES = TGGATES.drop(['Duration2','Duration unit'], axis=1)
TGGATES = TGGATES.sort_values(by=['InChI Key'])
TGGATES

,,dataset,Compound,InChI Key,CAS,Organism,Organ,Study type,Dose,Dosing,Duration
id,version,,,,,,,,,,
7285e2a4-dd0d-4585-83b6-a258ed097042,1,<PublishedDataset '7285e2a4-dd0d-4585-83b6-a258ed097042':1 - TG-GATES-gemfibrozil_Rat_Liver_in vivo_Repeat_15_day_high_FOLD_CHANGES>,gemfibrozil,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,Rat,Liver,in_vivo,high,Repeat,15day
cb29047e-7d5f-45a2-94ab-510406abc89e,1,<PublishedDataset 'cb29047e-7d5f-45a2-94ab-510406abc89e':1 - TG-GATES-gemfibrozil_Rat_Liver_in vivo_Repeat_29_day_low_FOLD_CHANGES>,gemfibrozil,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,Rat,Liver,in_vivo,low,Repeat,29day
44307c2a-ee3d-4d01-9ef2-4551149eb528,1,<PublishedDataset '44307c2a-ee3d-4d01-9ef2-4551149eb528':1 - TG-GATES-gemfibrozil_Rat_Liver_in vivo_Single_24_hr_high_FOLD_CHANGES>,gemfibrozil,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,Rat,Liver,in_vivo,high,Single,24hr
78400e01-a092-45be-9bbd-8365b924383f,1,<PublishedDataset '78400e01-a092-45be-9bbd-8365b924383f':1 - TG-GATES-gemfibrozil_Rat_Liver_in vitro_2_hr_middle_FOLD_CHANGES>,gemfibrozil,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,Rat,Liver,in_vitro,middle,None,2hr
d9fbe763-8eb6-431c-97a9-6fc44307d27a,1,<PublishedDataset 'd9fbe763-8eb6-431c-97a9-6fc44307d27a':1 - TG-GATES-gemfibrozil_Human_Liver_in vitro_2_hr_high_FOLD_CHANGES>,gemfibrozil,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,Human,Liver,in_vitro,high,None,2hr
...,...,...,...,...,...,...,...,...,...,...,...
efeb9b4d-4315-4e56-b8a3-151bb4a79e77,1,<PublishedDataset 'efeb9b4d-4315-4e56-b8a3-151bb4a79e77':1 - TG-GATES-fenofibrate_Rat_Liver_in vivo_Single_9_hr_low_FOLD_CHANGES>,fenofibrate,InChIKey=YMTINGFKWWXKFG-UHFFFAOYSA-N,49562-28-9,Rat,Liver,in_vivo,low,Single,9hr
96834324-4a34-4314-bab4-a330cf4cefcd,1,<PublishedDataset '96834324-4a34-4314-bab4-a330cf4cefcd':1 - TG-GATES-fenofibrate_Rat_Liver_in vivo_Single_24_hr_middle_FOLD_CHANGES>,fenofibrate,InChIKey=YMTINGFKWWXKFG-UHFFFAOYSA-N,49562-28-9,Rat,Liver,in_vivo,middle,Single,24hr
361abe50-9559-4033-b614-fc913a26b7ae,1,<PublishedDataset '361abe50-9559-4033-b614-fc913a26b7ae':1 - TG-GATES-fenofibrate_Rat_Liver_in vitro_2_hr_high_FOLD_CHANGES>,fenofibrate,InChIKey=YMTINGFKWWXKFG-UHFFFAOYSA-N,49562-28-9,Rat,Liver,in_vitro,high,None,2hr


## Filtering the datasets

In most cases, TG-GATES contains multiple datasets for the chemicals of interest, and should therefore be filtered. The next section allows the filtering of the datasets by the different columns from the TGGATES table. 

### Identify possible values per category
First, all possible values for filters should be identified, summarizing the TGGATES table for the columns:
- Compound
- Organism
- Organ
- Study type
- Dosing
- Dose
- Duration

Second, a table will be generated with the possible compounds and their corresponding CAS IDs and InChI Keys. 

In [15]:
#Options for categories
compounds = set(TGGATES['Compound'].tolist())
organisms = set(TGGATES['Organism'].tolist())
organs = set(TGGATES['Organ'].tolist())
studytypes = set(TGGATES['Study type'].tolist())
dosings = set(TGGATES['Dosing'].tolist())
doses = set(TGGATES['Dose'].tolist())
durations = set(TGGATES['Duration'].tolist())

print('Data available for compounds: '+str(compounds))
print('Data available for organisms: '+str(organisms))
print('Data available for organs: '+str(organs))
print('Data available for study types: '+str(studytypes))
print('Data available for dosings: '+str(dosings))
print('Data available for doses: '+str(doses))
print('Data available for durations: '+str(durations))

#Table for compounds and corresponding identifiers
df = pd.DataFrame()
chemdict = {}

for index, row in TGGATES.iterrows():
    if not row['Compound'] in chemdict:
        chemdict[row['Compound']] = {}
        chemdict[row['Compound']]['CAS'] = row['CAS']
        chemdict[row['Compound']]['InChI Key'] = row['InChI Key']

df = pd.DataFrame.from_dict(chemdict, orient='index')
df

Data available for compounds: {'gemfibrozil', 'fenofibrate', 'WY-14643', 'clofibrate'}
Data available for organisms: {'Rat', 'Human'}
Data available for organs: {'Kidney', 'Liver'}
Data available for study types: {'in_vivo', 'in_vitro'}
Data available for dosings: {'Repeat', None, 'Single'}
Data available for doses: {'low', 'middle', 'high'}
Data available for durations: {'3hr', '24hr', '6hr', '15day', '8hr', '4day', '9hr', '8day', '2hr', '29day'}


,CAS,InChI Key
gemfibrozil,25812-30-0,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N
clofibrate,637-07-0,InChIKey=KNHUKKLJHYUCFP-UHFFFAOYSA-N
WY-14643,50892-23-4,InChIKey=SZRPDCCEHVWOJX-UHFFFAOYSA-N
fenofibrate,49562-28-9,InChIKey=YMTINGFKWWXKFG-UHFFFAOYSA-N


### Filter the datasets
The values identified for each column can be used to filter the datasets. 

In order to do that, two lists should be filled:
- A list of all categories involved in the filter, corresponding to the column header. This list is called ```list_cat```
- A list of the filter values for all categories included in the category list. This list is called ```list_input```

Note that the location of the filled values is important. The sequence of the categories, and their values should correspond between the two lists. For each category, one filter value can be filled in the filter list. One can enter as many filters as necessary, from filtering for only the organism variable to filtering for all possible variables captured in the TGGATES dataframe. The ```res_df``` table contains the dataset(s) that are used later for pathway analysis.

In [16]:
list_cat = ["Compound","Organism","Organ","Study type","Dosing","Dose","Duration"]
list_input = ["gemfibrozil","Rat","Liver","in_vivo","Repeat","high","8day"]

filterlist = pd.DataFrame(
    {'Category': list_cat,
     'Input': list_input
    })
display(filterlist)

def tg_gates(df, list_cat, list_input):
    comb = zip(list_cat, list_input)
    sub_df = df
    for tup in comb:
        sub_df = sub_df[sub_df[tup[0]]==tup[1]]
    return sub_df

res_df = tg_gates(TGGATES, list_cat, list_input)
display(res_df)

,Category,Input
0,Compound,gemfibrozil
1,Organism,Rat
2,Organ,Liver
3,Study type,in_vivo
4,Dosing,Repeat
5,Dose,high
6,Duration,8day


,,dataset,Compound,InChI Key,CAS,Organism,Organ,Study type,Dose,Dosing,Duration
id,version,,,,,,,,,,
e45c8706-010e-4597-b490-df138f57f06e,1,<PublishedDataset 'e45c8706-010e-4597-b490-df138f57f06e':1 - TG-GATES-gemfibrozil_Rat_Liver_in vivo_Repeat_8_day_high_FOLD_CHANGES>,gemfibrozil,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,Rat,Liver,in_vivo,high,Repeat,8day


In [22]:
deg = []
for index, row in res_df.iterrows():
    file = row['dataset'].get_data(limit=100000)
    assay_deg = file[((file['logFC'] > 1) | (file['logFC'] < -1)) & (file['P.Value'] < 0.05) ]['ENTREZID'].unique().tolist()
    if len(assay_deg) > 0:
        for gene in assay_deg:
            deg.append(gene)
print('Number of differentially expressed genes: '+str(len(deg)))
degforpwanalysis = set(deg)

Number of differentially expressed genes: 50


# WikiPathways RDF

## Service description
WikiPathways is a community-driven molecular pathway database, supporting wide-spread topics and supported by many databases and integrative resources. It contains semantic annotations in its pathways for genes, proteins, metabolites, and interactions using a variety of reference databases, and WikiPathways is used to analyze and integrate experimental omics datasets ([Slenter et al., 2017](https://doi.org/10.1093/nar/gkx1064)). Furthermore, human pathways from Reactome ([Fabregat et al., 2018](https://doi.org/10.1093/nar/gkx1132)), another molecular pathway database, are integrated with WikiPathways and are therefore part of the WikiPathways RDF ([Waagmeester et al., 2016](https://doi.org/10.1371/journal.pcbi.1004989)). On the OpenRiskNet e-infrastructure, the WikiPathways RDF, which includes the Reactome pathways, is exposed via a Virtuoso SPARQL endpoint.

## Implementation
The first section is to find all molecular pathways in WikiPathways that contain the genes of interest, by matching the results of a SPARQL query to the list of genes found with the AOP-DB RDF. The SPARQL query extracts all Entrez gene IDs from all pathways in WikiPathways. When overlap between those lists are found, the pathway ID is stored in a dataframe along with its title, and organism.

The next section is for extracting all pathways for the species of interest, along with all genes present. These are used later for pathway analysis with the data extracted from TG-GATES through the EdelweissData explorer.

In [23]:
sparqlquery = '''
    SELECT DISTINCT (str(?wpid) as ?Pathway_ID) (str(?PW_Title) as ?Pathway_title) (fn:substring(?ncbiGeneId,33) as ?Entrez)  ?organism
    WHERE {
    ?gene a wp:GeneProduct; dcterms:identifier ?id; dcterms:isPartOf ?pathwayRes; wp:bdbEntrezGene ?ncbiGeneId.
    ?pathwayRes a wp:Pathway; dcterms:identifier ?wpid; dc:title ?PW_Title; wp:organismName ?organism.}
    '''
wikipathwayssparql.setQuery(sparqlquery)
wikipathwayssparql.setReturnFormat(JSON)  
results = wikipathwayssparql.query().convert()

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

WikiPathwaysGenes = {}
WikiPathwaysNames = {}
WikiPathwaysOrganism = {}
for result in results["results"]["bindings"]:
    WikiPathwaysGenes[result["Pathway_ID"]["value"]] = set([])
for result in results["results"]["bindings"]:
    WikiPathwaysGenes[result["Pathway_ID"]["value"]].add(result["Entrez"]["value"])
for result in results["results"]["bindings"]:
    WikiPathwaysNames[result["Pathway_ID"]["value"]] = result["Pathway_title"]["value"]
for result in results["results"]["bindings"]:
    WikiPathwaysOrganism[result["Pathway_ID"]["value"]] = result["organism"]["value"]


for lst in WikiPathwaysGenes:
    genematch = intersection(WikiPathwaysGenes[lst], Genes)
    WikiPathwaysGenes[lst] = [WikiPathwaysNames[lst], WikiPathwaysOrganism[lst], genematch, len(genematch)]

WPtable = pd.DataFrame.from_dict(WikiPathwaysGenes,orient='index',columns=['Pathway_title','Organism','Entrez Gene','nGenes'])
WPtable = WPtable[WPtable.nGenes >= 1]
WPtable = WPtable.drop(columns='nGenes')
display(WPtable)

,Pathway_title,Organism,Entrez Gene
WP1797,Circadian Clock,Homo sapiens,[5465]
WP3370,RORA activates gene expression,Homo sapiens,[5465]
WP299,Nuclear Receptors in Lipid Metabolism and Toxicity,Homo sapiens,[5465]
WP4721,Eicosanoid metabolism via Lipo Oxygenases (LOX),Homo sapiens,[5465]
WP4447,SUMOylation of intracellular receptors,Homo sapiens,[5465]
WP4720,Eicosanoid metabolism via Cytochrome P450 Mono-Oxygenases (CYP) pathway,Homo sapiens,[5465]
WP2011,SREBF and miR33 in cholesterol and lipid homeostasis,Homo sapiens,[5465]
WP2882,Nuclear Receptors Meta-Pathway,Homo sapiens,[5465]
WP3594,Circadian rhythm related genes,Homo sapiens,[5465]
WP4396,Nonalcoholic fatty liver disease,Homo sapiens,[5465]


## Extracting patways for pathway analysis

Prior to extracting pathways, the species of interest should be stored in a variable. Note that this should be the latin species name, and corresponding to the filtered dataset(s) from TG-GATES.

In [24]:
OrganismFilter = 'Rattus norvegicus'

In [25]:
sparqlquery = '''
    SELECT DISTINCT (str(?wpid) as ?Pathway_ID) (str(?PW_Title) as ?Pathway_title) (fn:substring(?ncbiGeneId,33) as ?Entrez) 
    WHERE {
    ?gene a wp:GeneProduct; dcterms:identifier ?id; dcterms:isPartOf ?pathwayRes; wp:bdbEntrezGene ?ncbiGeneId.
    ?pathwayRes a wp:Pathway; dcterms:identifier ?wpid; dc:title ?PW_Title; wp:organismName "'''+OrganismFilter+'''"^^xsd:string.}
    '''
wikipathwayssparql.setQuery(sparqlquery)
wikipathwayssparql.setReturnFormat(JSON)  
results = wikipathwayssparql.query().convert()

WikiPathwaysGenes = {}
WikiPathwaysNames = {}
for result in results["results"]["bindings"]:
    WikiPathwaysGenes[result["Pathway_ID"]["value"]] = set([])
for result in results["results"]["bindings"]:
    WikiPathwaysNames[result["Pathway_ID"]["value"]] = result["Pathway_title"]["value"]
for result in results["results"]["bindings"]:
    WikiPathwaysGenes[result["Pathway_ID"]["value"]].add(result["Entrez"]["value"])

for lst in WikiPathwaysGenes:
    WikiPathwaysGenes[lst] = [WikiPathwaysGenes[lst],len(WikiPathwaysGenes[lst])]

WPtable = pd.DataFrame.from_dict(WikiPathwaysGenes,orient='index',columns=['Genes','nGenes'])
display(WPtable)

,Genes,nGenes
WP1286,"{154516, 24861, 292155, 25279, 301264, 29738, 311257, 103690051, 302302, 24424, 494499, 81869, 361631, 316325, 574523, 396551, 293779, 116631, 292915, 25426, 65030, 246767, 65185, 24426, 192242, 286954, 305264, 304322, 25147, 24404, 307092, 113992, 299566, 308190, 114846, 108348061, 246245, 24422, 24192, 24902, 26760, 685402, 301517, 100910526, 310848, 24912, 500257, 361510, 295430, 353498, 81676, 25458, 500892, 363618, 310855, 171341, 154985, 29326, 297029, 286921, 58953, 116632, 246247, 24298, 500359, 116686, 100910462, 303218, 293451, 25315, 312495, 84406, 364476, 25428, 83783, 288108, 24296, 114700, 301595, 81924, 360268, 499302, 64352, 171445, 396527, 290623, 300850, 307838, 25086, 25355, 25427, 25146, 29725, 286989, 303669, 289197, 691394, 362228, 24297, 108348148, ...}",143
WP1290,"{24508, 24577, 81525, 84359, 29884, 502004, 64547, 24185, 291100, 103694380, 500592, 309165, 100911660, 100360940, 309295, 117279, 25718, 60434, 292892, 364081, 362675, 492821, 25272, 296953, 78963, 687813, 78971, 25166, 83584, 25402, 24842, 316256, 64625, 290749, 114555, 314856, 25385, 312030, 64044, 140923, 24482, 309361, 25008, 64026, 24516, 116502, 314756, 156767, 25625, 103689977, 64314, 266610, 246775, 103690372, 287398, 246097, 362788, 116667, 64041, 294071, 246756, 24887, 246334, 25513, 84351, 316241, 24483, 25493, 60371, 114214, 64639, 63879, 24224, 293624, 311786, 58918, 81736, 306886}",78
WP1278,"{84351, 309361, 313121, 116554, 116590, 309452, 29538, 311245, 103694380, 81736, 309165, 81780, 299331, 24481, 360640, 286908, 246756}",17
WP1279,"{84581, 288588, 363481, 498109, 116590, 24185, 288533, 690966, 81649, 84577, 297893, 81504, 50658, 292778, 114486, 314322, 84582, 103695118, 81646, 58919, 100363500, 84578, 363287, 84580, 170922, 313845, 306516, 24400, 81674, 83503, 83828, 310784, 117526, 680149, 361580, 682902, 497672, 171150, 307485, 309361, 171104, 117017, 24790, 24516, 25636, 288651, 294236, 81673, 83805, 289561, 294018, 361365, 309224, 294693, 103690054, 291703, 308415, 84351, 363633, 24890, 24224, 170851, 84389, 293621, 367858, 373541, 314612, 314436, 54244, 81736, 170915, 363067, 266713, 303918}",74
WP1282,"{368066, 25331, 690050, 300691, 689330, 171347, 24661, 24267, 81676}",9
...,...,...
WP547,"{85253, 306761, 24233, 79224, 24946, 25439, 113959, 25619, 24548, 24903, 83580, 24366, 298566, 81750, 116669, 117512, 29251, 25048, 29243, 113936, 295703, 65051, 79126, 288001, 289055, 29333, 29436, 117517, 302470, 25268, 289395, 50692, 24232, 24648, 54249, 24153, 29687, 192262, 313421, 64036, 64459, 24234, 362634, 304917, 155012, 24231, 25584, 287527, 81509, 24441, 25407, 312705, 24617, 24237, 54243, 290757, 84007, 64023, 260320, 25692, 294257}",61
WP505,"{25125, 84353, 317376, 50554, 25712, 161452, 59328, 59107, 50658, 25313, 314322, 59086, 85435, 25495, 114208, 24881, 60584, 300054, 311061, 156726, 445442, 29357, 25631, 25296, 24835, 103691556, 311071, 24516, 24373, 81516, 94188, 367264, 25671, 29200, 50689, 25353, 316742, 367218, 25639, 367100, 29591, 83837, 29610, 293621, 367858, 54244, 24617, 81736, 81810, 25124, 170915, 313477, 497010, 84598}",54
WP654,"{300711, 25203, 114483, 54237, 25112, 58919, 492821, 78963, 24708, 25402, 24842, 497672, 311562, 116502, 399489, 680110, 114851, 300668, 25729, 24887, 114212, 25309, 362817, 24224, 58918, 298795, 100363502}",27
WP89,"{24493, 24471, 64159, 84359, 362456, 117279, 140657, 299625, 313121, 24708, 78963, 116554, 83584, 25402, 29432, 25385, 64044, 24835, 140926, 64026, 24516, 266610, 25591, 103689977, 289014, 287398, 246097, 362491, 315994, 116667, 360748, 114214, 25309, 24224, 58918, 60374, 116685, 100363502, 29431}",39


This section is the actual pathway analysis, using the differentially expressed genes from TG-GATES, and the molecular pathways from WikiPathways. With some basic statistics, a Z-score can be calculated for all pathways. A Z-score above 1.96 is considered significant.

In [26]:
ngenepres = []
for index, row in WPtable.iterrows():
    genepres = []
    for gene in row['Genes']:
        for sig in degforpwanalysis:
            if gene == str(sig):
                genepres.append(gene)
    ngenepres.append(len(genepres))
WPtable['nSigGenes'] = ngenepres
WPtable['percentSigGenes'] = (WPtable['nSigGenes'] / WPtable['nGenes'])*100

total = []
for index, row in WPtable.iterrows():
    total.append(row['nSigGenes'])

StandardDeviation = statistics.stdev(total)
ExpectedValue = (sum(total)/len(WPtable))

WPtable['Zscore'] = (WPtable['nSigGenes'] - ExpectedValue)/StandardDeviation
WPtable = WPtable.sort_values(by=['Zscore'], ascending=False)
WPtable = WPtable[WPtable.Zscore >= 1.96]
display(WPtable)

,Genes,nGenes,nSigGenes,percentSigGenes,Zscore
WP1286,"{154516, 24861, 292155, 25279, 301264, 29738, 311257, 103690051, 302302, 24424, 494499, 81869, 361631, 316325, 574523, 396551, 293779, 116631, 292915, 25426, 65030, 246767, 65185, 24426, 192242, 286954, 305264, 304322, 25147, 24404, 307092, 113992, 299566, 308190, 114846, 108348061, 246245, 24422, 24192, 24902, 26760, 685402, 301517, 100910526, 310848, 24912, 500257, 361510, 295430, 353498, 81676, 25458, 500892, 363618, 310855, 171341, 154985, 29326, 297029, 286921, 58953, 116632, 246247, 24298, 500359, 116686, 100910462, 303218, 293451, 25315, 312495, 84406, 364476, 25428, 83783, 288108, 24296, 114700, 301595, 81924, 360268, 499302, 64352, 171445, 396527, 290623, 300850, 307838, 25086, 25355, 25427, 25146, 29725, 286989, 303669, 289197, 691394, 362228, 24297, 108348148, ...}",143,4,2.797203,5.401926
WP1307,"{170670, 113976, 117243, 25330, 25413, 25288, 364975, 25062, 24849, 94340, 361676, 25014, 117035, 171155, 140547, 24539, 311569, 79223, 311849, 24158, 24538, 289481, 113965, 29367, 117543, 25757, 25287, 114024, 29740, 100911615, 25363, 298942, 64304, 25756, 50682}",35,3,8.571429,3.959527
WP506,"{170670, 291468, 113976, 25330, 117243, 25413, 25288, 364975, 25062, 94340, 361676, 25014, 117035, 171155, 24539, 100911186, 311569, 311849, 24158, 289481, 24538, 113965, 29367, 117543, 25757, 25287, 114024, 29740, 100911615, 298942, 25363, 64304, 25756, 50682}",34,3,8.823529,3.959527
WP1297,"{29563, 25271, 366791, 310903, 499985, 25703, 24172, 89826, 24705, 29184, 24539, 25086, 361801, 29646, 690953, 83574, 24706, 266603, 100145871, 293049, 25061, 154985, 25056, 155192, 24188, 313689, 116676, 362662, 685072, 246298, 114628, 292915, 432367, 24710, 64047, 25073, 353252, 312495, 100365047, 314264, 83783, 114106}",42,3,7.142857,3.959527
WP419,"{170670, 24158, 25363, 113976, 171142, 113965, 117035, 25413, 25288, 64304, 25757, 25287, 114024, 25541, 29740, 113956}",16,3,18.750000,3.959527
WP139,"{114700, 84356, 24646, 25270, 81924, 25279, 25664, 313210, 24307, 24891, 25303, 24705, 25086, 83569, 24706, 25682, 65035, 85264, 361523, 154985, 140668, 84385, 685072, 24297, 29277, 114628, 60351, 24873, 58852, 170913, 25428, 25747}",32,2,6.250000,2.517128
WP145,"{25384, 24538, 84497, 25675, 300438, 25292, 310900, 25428, 25728, 296371, 25081, 24539, 25073, 24207, 25080, 299858, 81782, 313210, 108348160, 24530}",20,2,10.000000,2.517128
WP2376,"{116643, 140727, 85420, 100361457, 116590, 684969, 298947, 29224, 83472, 50658, 24451, 116554, 24424, 310392, 291796, 100158233, 81869, 497672, 24919, 29437, 366960, 29739, 24565, 85421, 24534, 287876, 50689, 497931, 304127, 140668, 24314, 29292, 24426, 361632, 295549, 83688, 25445, 25522, 25150, 58960, 24185, 114846, 108348061, 26760, 114495, 25365, 64188, 24842, 100360087, 679217, 25581, 171379, 25458, 24516, 24252, 171341, 114851, 286921, 85430, 361568, 29326, 297029, 117262, 57298, 116667, 25513, 301252, 301555, 117254, 116686, 170538, 24778, 170851, 25283, 293621, 54349, 54244, 25315, 681050, 84027, 25073, 170915, 83619, 117263, 305540, 24552, 24189, 25023, 29741, 25260, 497932, 24908, 64191, 300711, 100912585, 81649, 289623, 25352, 79255, 65052, ...}",167,2,1.197605,2.517128
WP372,"{170670, 113976, 117243, 25330, 25413, 25288, 364975, 25062, 24849, 94340, 361676, 25014, 117035, 171155, 140547, 24539, 311569, 79223, 311849, 24158, 24538, 289481, 113965, 29367, 25757, 25287, 114024, 100911615, 25363, 298942, 64304, 25756, 50682}",33,2,6.060606,2.517128


In [33]:
SigPathways = list(WPtable.index)
for WP in SigPathways:
    print(WP + '\t' +WikiPathwaysNames[WP])
print('\nBased on dataset(s): ')
display(res_df)

WP1286	Metapathway biotransformation
WP1307	Fatty Acid Beta Oxidation
WP506	Fatty Acid Beta Oxidation
WP1297	Retinol metabolism
WP419	Mitochondrial LC-Fatty Acid Beta-Oxidation
WP139	Nuclear receptors in lipid metabolism and toxicity
WP145	Statin Pathway
WP2376	Nuclear factor, erythroid-derived 2, like 2 signaling pathway
WP372	Beta Oxidation Meta Pathway

Based on dataset(s): 


,,dataset,Compound,InChI Key,CAS,Organism,Organ,Study type,Dose,Dosing,Duration
id,version,,,,,,,,,,
e45c8706-010e-4597-b490-df138f57f06e,1,<PublishedDataset 'e45c8706-010e-4597-b490-df138f57f06e':1 - TG-GATES-gemfibrozil_Rat_Liver_in vivo_Repeat_8_day_high_FOLD_CHANGES>,gemfibrozil,InChIKey=HEMJJKBWTPKOJG-UHFFFAOYSA-N,25812-30-0,Rat,Liver,in_vivo,high,Repeat,8day


In [28]:
%load_ext watermark

#python, ipython, packages, and machine characteristics
%watermark -v -m -p sys,pip,SPARQLWrapper,pandas,json,re,requests,warnings,pyvis,matplotlib,numpy,IPython,urllib,seaborn,statistics

#dte
print(" ")
%watermark -u -n -t -z

CPython 3.6.3
IPython 7.9.0

sys 3.6.3 (default, May 31 2019, 13:05:43) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-36)]
pip 20.0.1
SPARQLWrapper 1.8.5
pandas 0.25.3
json 2.0.9
re 2.2.1
requests 2.22.0
warnings unknown
pyvis 0.1.7.0
matplotlib not installed
numpy 1.18.1
IPython 7.9.0
urllib unknown
seaborn not installed
statistics unknown

compiler   : GCC 4.8.5 20150623 (Red Hat 4.8.5-36)
system     : Linux
release    : 3.10.0-1062.1.2.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 60
interpreter: 64bit
 
last updated: Thu Jan 23 2020 14:00:14 UTC


In [31]:
sparqlquery = '''
SELECT ?originaldata 
WHERE{
?dataset a void:Dataset ; 
pav:createdWith ?originaldata .
}'''
aopwikisparql.setQuery(sparqlquery)
aopwikisparql.setReturnFormat(JSON)  
results = aopwikisparql.query().convert()

Result = results["results"]["bindings"][0]['originaldata']["value"]

print("The underlying dataset for AOP-Wiki RDF: "+Result)

The underlying dataset for AOP-Wiki RDF: aop-wiki-xml-2019-07-01


In [35]:
sparqlquery = '''
select distinct ?dataset ?date where {
  ?dataset a void:Dataset ;
  pav:createdOn ?date .
}'''
wikipathwayssparql.setQuery(sparqlquery)
wikipathwayssparql.setReturnFormat(JSON)  
results = wikipathwayssparql.query().convert()

Result = results["results"]["bindings"][0]['dataset']["value"]
Date = results["results"]["bindings"][0]['date']["value"]

print("The WikiPathways RDF used in this notebook: "+Result+ " created on "+ Date)

The WikiPathways RDF used in this notebook: http://data.wikipathways.org/20191210/rdf/ created on 2019-12-09T23:28:23.591Z
